In [1]:
import os 
import duckdb
import pandas as pd
import numpy as np

In [2]:
con = duckdb.connect(database=':memory:', read_only=False)
con.sql("INSTALL spatial;")
con.sql("INSTALL json;")

con.sql("LOAD spatial;")
con.sql("LOAD json;")

In [3]:
# create schema and set defaults 
con.sql("CREATE SCHEMA analytics;")

In [4]:
# create the neighborhood table

neighborhood_table = """
CREATE OR REPLACE TABLE analytics.neighborhood_geo_data AS
WITH raw_data AS (
    -- Reads the GeoJSON file and converts it into a nested table structure
    SELECT * FROM read_json_auto('source-files/Neighbourhoods.geojson')
),
unnested_features AS (
    -- Flattens the 'features' array into individual rows
    SELECT unnest(raw_data.features) AS feature_struct FROM raw_data
)
SELECT
    -- 1. typedefs 
    feature_struct.properties._id,
    feature_struct.properties.AREA_ID::BIGINT AS AREA_ID,
    feature_struct.properties.AREA_ATTR_ID::BIGINT AS AREA_ATTR_ID,
    feature_struct.properties.PARENT_AREA_ID,
    feature_struct.properties.AREA_SHORT_CODE::VARCHAR AS AREA_SHORT_CODE,
    feature_struct.properties.AREA_LONG_CODE::VARCHAR AS AREA_LONG_CODE,
    feature_struct.properties.AREA_NAME::VARCHAR AS AREA_NAME,
    feature_struct.properties.AREA_DESC,
    feature_struct.properties.CLASSIFICATION::VARCHAR AS CLASSIFICATION,
    feature_struct.properties.CLASSIFICATION_CODE::VARCHAR AS CLASSIFICATION_CODE,
    feature_struct.properties.OBJECTID::DOUBLE AS OBJECTID,
    
    -- 2. Convert the GeoJSON geometry into a WKT string
    --    FIX: Use to_json() to convert the STRUCT to a valid JSON string
    ST_AsText(ST_GeomFromGeoJSON(to_json(feature_struct.geometry))) AS geometry_wkt,
    
    -- 3. Keep the geometry type for reference (simplified access)
    feature_struct.geometry.type AS geometry_type
FROM unnested_features;
"""

con.sql(neighborhood_table)

In [5]:
# Check the table structure and data types
con.sql("PRAGMA table_info('analytics.neighborhood_geo_data')").show()

# Alternatively:
# con.sql("DESCRIBE analytics.neighborhood_geo_data").show()

┌───────┬─────────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │        name         │  type   │ notnull │ dflt_value │   pk    │
│ int32 │       varchar       │ varchar │ boolean │  varchar   │ boolean │
├───────┼─────────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ _id                 │ BIGINT  │ false   │ NULL       │ false   │
│     1 │ AREA_ID             │ BIGINT  │ false   │ NULL       │ false   │
│     2 │ AREA_ATTR_ID        │ BIGINT  │ false   │ NULL       │ false   │
│     3 │ PARENT_AREA_ID      │ JSON    │ false   │ NULL       │ false   │
│     4 │ AREA_SHORT_CODE     │ VARCHAR │ false   │ NULL       │ false   │
│     5 │ AREA_LONG_CODE      │ VARCHAR │ false   │ NULL       │ false   │
│     6 │ AREA_NAME           │ VARCHAR │ false   │ NULL       │ false   │
│     7 │ AREA_DESC           │ VARCHAR │ false   │ NULL       │ false   │
│     8 │ CLASSIFICATION      │ VARCHAR │ false   │ NULL       │ false   │
│     9 │ CLASSIFICATION_

In [6]:
# View the first 5 rows and focus on the WKT
con.sql("""
    SELECT 
        AREA_NAME, 
        geometry_type, 
        LEFT(geometry_wkt, 60) AS geometry_wkt_sample, 
        LENGTH(geometry_wkt) AS wkt_length
    FROM analytics.neighborhood_geo_data 
    LIMIT 5
""").show()

┌───────────────────────────┬───────────────┬──────────────────────────────────────────────────────────────┬────────────┐
│         AREA_NAME         │ geometry_type │                     geometry_wkt_sample                      │ wkt_length │
│          varchar          │    varchar    │                           varchar                            │   int64    │
├───────────────────────────┼───────────────┼──────────────────────────────────────────────────────────────┼────────────┤
│ South Eglinton-Davisville │ MultiPolygon  │ MULTIPOLYGON (((-79.3863510515018 43.6978312650188, -79.3862 │       1949 │
│ North Toronto             │ MultiPolygon  │ MULTIPOLYGON (((-79.3974366551459 43.7069299169967, -79.3983 │       1018 │
│ Dovercourt Village        │ MultiPolygon  │ MULTIPOLYGON (((-79.4341131654386 43.660145276359, -79.43536 │       2766 │
│ Junction-Wallace Emerson  │ MultiPolygon  │ MULTIPOLYGON (((-79.4387000029275 43.6676608052133, -79.4384 │       6104 │
│ Yonge-Bay Corridor    

In [7]:
con.sql("SELECT COUNT(*) FROM analytics.neighborhood_geo_data").show() # should be 158 rows 

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          158 │
└──────────────┘



In [8]:
# make the PK using neighborhood lists 

# Trinity-Bellwoods, West Queen West Area (West-Central Core)
tor_c01 = ['Dufferin Grove', 'Palmerston-Little Italy', 'Trinity-Bellwoods', 'University', 
           'Kensington-Chinatown', 
           'West Queen West'] # **ADDED**
# NOTE: Wellington Place is moved to c08 for closer fit with core/waterfront

tor_c02 = ['Wychwood', 'Casa Loma', 'Annex', 'Yonge-St.Clair']
tor_c03 = ['Humewood-Cedarvale', 'Forest Hill South', 'Yonge-Eglinton']

# Uptown (Lawrence Park Area)
tor_c04 = ['Bedford Park-Nortown', 'Lawrence Park North', 'Lawrence Park South', 
           'Englemount-Lawrence', 'Forest Hill North',
           'North Toronto'] # **ADDED**
           
tor_c06 = ['Bathurst Manor', 'Clanton Park']
tor_c07 = ['Newtonbrook West', 'Newtoonbrook East', 'Willowdale West', 'Lansing-Westgate']

# Downtown Core, Waterfront, East Core (The main hub)
tor_c08 = ['North St.James Town', 'Cabbagetown-South St.James Town', 'Regent Park', 
           'Moss Park', 'Yonge-Bay Corridor', 
           'Bay-Cloverhill', 'Downtown Yonge East', 'Church-Wellesley', 
           'St Lawrence-East Bayfront-The Islands', 'Harbourfront-CityPlace', 
           'Wellington Place'] # **ADDED** (All Core/Waterfront)

# Rosedale Area
tor_c09 = ['Rosedale-Moore Park', 
           'Mount Pleasant East'] # **ADDED**

# Midtown North (Yonge/Eglinton Area)
tor_c10 = ['Mount Pleasant West', 
           'South Eglinton-Davisville'] # **ADDED**
           
tor_c11 = ['Flemingdon Park', 'Thorncliffe Park', 'Leaside-Bennington']
tor_c12 = ['St.Andrew-Windfields', 'Bridle Path-Sunnybrook-York Mills']
tor_c13 = ["Parkwoods-O'Connor Hills", "Banbury-Don Mills", "Victoria Village", "Fenside-Parkwoods"]

# North York Yonge Corridor (Closest to Avondale/Yonge-Doris)
tor_c14 = ['NewtonBrook East', 'East Willowdale', 
           'Avondale', 'Yonge-Doris'] # **ADDED**
           
tor_c15 = ['Bayview Village', 'Don Valley Village', 'Hillcrest Village', 'Bayview Woods-Steeles', 'Pleasant View', 'Henry Farm']
# missing islands and waterfront community 

# Far East Scarborough (Rouge/Malvern Area)
tor_e11 = ['West Rouge', 'Malvern East', 'Malvern West',
           'Morningside Heights'] # **ADDED**
           
tor_e10 = ['Highland Creek', 'Centennial Scarborough', 'West Hill']
tor_e09 = ['Morningside', 'Woburn North', 'Golfdale-Cedarbrae-Woburn', 'Bendale South', 'Bendale-Glen Andrew']
tor_e08 = ['Guildwood', 'Scarborough Village', 'Cliffcrest', 'Eglinton East']
tor_e07 = ['Milliken', 'Agincourt North', 'Agincourt South-Malvern West']

# Scarborough/North York East (L'Amoreaux Area)
tor_e05 = ["Steeles", "East L'Amoreaux", "L'Amoreaux West", "Tam O'Shanter-Sullivan",
           'Newtonbrook East'] # **ADDED**

tor_e06 = ['Oakridge', 'Birchcliffe-Cliffside']

# East York (O'Connor Area)
tor_e03 = ["O'Connor-Parkview", "Old East York", "Danforth East York", "Danforth", "Broadview North", 
           "Playter Estates-Danforth", "Woodbine-Lumsden",
           'Taylor-Massey'] # **ADDED**
           
tor_e02 = ['The Beaches', 'East End-Danforth', 'Woodbine Corridor', 'Greenwood-Coxwell']
tor_e01 = ['North Riverdale', 'Blake-Jones', 'South Riverdale']
tor_e04 = ['Wexford/Maryvale', 'Dorset Park', 'Ionview', 'Kennedy Park', 'Clairlea-Birchmount']

# Etobicoke North
tor_w10 = ['West Humber-Clairville', 'Mount Olive-Silverstone-Jamestown', 'Rexdale-Kipling', 'Elms-Old Rexdale',
           'Thistletown-Beaumond Heights'] # **ADDED**

# Etobicoke Centre
tor_w09 = ['Kingsview Village-The Westway', 'Humber Heights-Westmount', 
           'Wilowridge-Martingrove-Richview',
           'Willowridge-Martingrove-Richview'] # **ADDED**

# Etobicoke Centre/Lakeshore (Islington Area)
tor_w08 = ['Eringate-Centennial-West Deane', 'Markland Wood', 'Princess-Rosethorn', 
           'Edenbridge-Humber Valley', 'Kingsway South', 'Etobicoke West Mall', 
           'Etobicoke City Centre',
           'Islington'] # **ADDED**

tor_w07 = ['Stonegate-Queensway']

# Etobicoke Lakeshore (Humber Bay Area)
tor_w06 = ['Alderwood', 'Long Branch', 'New Toronto', 'Mimico-Queensway',
           'Humber Bay Shores'] # **ADDED**

# North York West (Jane/Finch & Downsview Areas)
tor_w05 = ['Humber Summit', 'Humbermede', 'Downsview', 
           'Glenfield-Jane Heights', 'Black Creek', 'Fort York-Liberty Village',
           'Oakdale-Beverley Heights', 'Westminster-Branson', 'York University Heights'] # **ADDED**

tor_w04 = ['Pelmo Park-Humberlea', 'Weston', 'Mount Dennis', 'Rustic', 
           'Maple Leaf', 'Yorkdale-Glen Park', 'Briar Hill-Belgravia', 
           'Brookhaven-Amesbury', 'Beechborough-Greenbrook']

# York South-Weston (Oakwood Area)
tor_w03 = ['Rockcliffe-Smythe', 'Keelesdale-Eglinton West', 'Caledonia-Fairbank', 
           'Corso Italia-Davenport', 'Weston-Pelham Park',
           'Oakwood Village'] # **ADDED**

# Parkdale-High Park (Junction Area)
tor_w02 = ['Lambton Baby Point', 'Runnymede-Bloor West Village', 'Junction Area', 
           'High Park North', 'Dovercourt Village',
           'Junction-Wallace Emerson'] # **ADDED**

# Parkdale-High Park (Little Portugal Area)
tor_w01 = ['High Park-Swansea', 'Roncesvalles', 'South Parkdale',
           'Little Portugal'] # **ADDED**

In [9]:
all_classif = [
    (tor_w01, 'Toronto W01'),
    (tor_w02, 'Toronto W02'),
    (tor_w03, 'Toronto W03'),
    (tor_w04, 'Toronto W04'),
    (tor_w05, 'Toronto W05'),
    (tor_w06, 'Toronto W06'),
    (tor_w07, 'Toronto W07'),
    (tor_w08, 'Toronto W08'),
    (tor_w09, 'Toronto W09'),
    (tor_w10, 'Toronto W10'),
    (tor_e01, 'Toronto E01'),
    (tor_e02, 'Toronto E02'),
    (tor_e03, 'Toronto E03'),
    (tor_e04, 'Toronto E04'),
    (tor_e05, 'Toronto E05'),
    (tor_e06, 'Toronto E06'),
    (tor_e07, 'Toronto E07'),
    (tor_e08, 'Toronto E08'),
    (tor_e09, 'Toronto E09'),
    (tor_e10, 'Toronto E10'),
    (tor_e11, 'Toronto E11'), 
    (tor_c01, 'Toronto C01'),
    (tor_c02, 'Toronto C02'),
    (tor_c03, 'Toronto C03'),
    (tor_c04, 'Toronto C04'),
    (tor_c06, 'Toronto C06'),
    (tor_c07, 'Toronto C07'),
    (tor_c08, 'Toronto C08'),
    (tor_c09, 'Toronto C09'),
    (tor_c10, 'Toronto C10'),
    (tor_c11, 'Toronto C11'),
    (tor_c12, 'Toronto C12'),
    (tor_c13, 'Toronto C13'),
    (tor_c14, 'Toronto C14'),
    (tor_c15, 'Toronto C15'),
]

print(all_classif)

[(['High Park-Swansea', 'Roncesvalles', 'South Parkdale', 'Little Portugal'], 'Toronto W01'), (['Lambton Baby Point', 'Runnymede-Bloor West Village', 'Junction Area', 'High Park North', 'Dovercourt Village', 'Junction-Wallace Emerson'], 'Toronto W02'), (['Rockcliffe-Smythe', 'Keelesdale-Eglinton West', 'Caledonia-Fairbank', 'Corso Italia-Davenport', 'Weston-Pelham Park', 'Oakwood Village'], 'Toronto W03'), (['Pelmo Park-Humberlea', 'Weston', 'Mount Dennis', 'Rustic', 'Maple Leaf', 'Yorkdale-Glen Park', 'Briar Hill-Belgravia', 'Brookhaven-Amesbury', 'Beechborough-Greenbrook'], 'Toronto W04'), (['Humber Summit', 'Humbermede', 'Downsview', 'Glenfield-Jane Heights', 'Black Creek', 'Fort York-Liberty Village', 'Oakdale-Beverley Heights', 'Westminster-Branson', 'York University Heights'], 'Toronto W05'), (['Alderwood', 'Long Branch', 'New Toronto', 'Mimico-Queensway', 'Humber Bay Shores'], 'Toronto W06'), (['Stonegate-Queensway'], 'Toronto W07'), (['Eringate-Centennial-West Deane', 'Markland

In [10]:
classif_df = pd.DataFrame(
    all_classif,
    columns=['AREA_NAME_KEY', 'Region_classif'] # none of the area name keys and regions are null so no checks needed 
)
classif_df.head()
con.register('classif_lookup', classif_df)

In [11]:
# left join for analytics.neighborhood_geo_data
final_classification_query = """
CREATE OR REPLACE TABLE analytics.neighborhood_classified AS
SELECT
    t1.*, -- Select all columns from the main neighborhood table
    t2_flat.Region_classif -- Get the classification ID
FROM analytics.neighborhood_geo_data AS t1
LEFT JOIN 
(
    -- CTE: Flatten the lookup table by unnesting the array column
    SELECT 
        unnest(AREA_NAME_KEY) AS Single_Area_Name, -- UNNEST converts the array into many rows
        Region_classif
    FROM classif_lookup
) AS t2_flat 
    -- Now, join the single area name to the main table's area name
    ON t1.AREA_NAME = t2_flat.Single_Area_Name;
"""

con.sql(final_classification_query)

# Verification Check
con.sql("SELECT AREA_NAME, Region_classif FROM analytics.neighborhood_classified WHERE Region_classif IS NOT NULL").show()

┌───────────────────────────────────┬────────────────┐
│             AREA_NAME             │ Region_classif │
│              varchar              │    varchar     │
├───────────────────────────────────┼────────────────┤
│ South Eglinton-Davisville         │ Toronto C10    │
│ North Toronto                     │ Toronto C04    │
│ Dovercourt Village                │ Toronto W02    │
│ Junction-Wallace Emerson          │ Toronto W02    │
│ Yonge-Bay Corridor                │ Toronto C08    │
│ Bay-Cloverhill                    │ Toronto C08    │
│ Bendale-Glen Andrew               │ Toronto E09    │
│ Downsview                         │ Toronto W05    │
│ Oakdale-Beverley Heights          │ Toronto W05    │
│ Avondale                          │ Toronto C14    │
│    ·                              │      ·         │
│    ·                              │      ·         │
│    ·                              │      ·         │
│ Kingsview Village-The Westway     │ Toronto W09    │
│ Elms-Old

In [12]:
# check for duplicates 
query = f"""
SELECT
    AREA_NAME,
    COUNT(AREA_NAME) AS duplicate_count
FROM analytics.neighborhood_classified
GROUP BY AREA_NAME
HAVING COUNT(AREA_NAME) > 1
ORDER BY duplicate_count DESC;
"""

con.sql(query).show()

┌───────────┬─────────────────┐
│ AREA_NAME │ duplicate_count │
│  varchar  │      int64      │
├───────────┴─────────────────┤
│           0 rows            │
└─────────────────────────────┘



In [13]:
# strategy: use the classif table and do left joins again with the target variable 

# load the rentals first
csv_load_query = """
CREATE OR REPLACE TABLE analytics.rental_table AS
SELECT * FROM read_csv_auto('source-files/rental_data_extraction.csv');
""" 

con.sql(csv_load_query)

join_query = """
CREATE OR REPLACE TABLE analytics.neighborhoods AS
SELECT
    t1.*, -- all the rows of t1 
    t2.*
FROM analytics.neighborhood_classified AS t1
LEFT JOIN 
    analytics.rental_table AS t2 
    ON t1.Region_classif = t2.Area
"""

con.sql(join_query)
con.sql("PRAGMA table_info('analytics.neighborhoods')").show()

┌───────┬───────────────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │           name            │  type   │ notnull │ dflt_value │   pk    │
│ int32 │          varchar          │ varchar │ boolean │  varchar   │ boolean │
├───────┼───────────────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ _id                       │ BIGINT  │ false   │ NULL       │ false   │
│     1 │ AREA_ID                   │ BIGINT  │ false   │ NULL       │ false   │
│     2 │ AREA_ATTR_ID              │ BIGINT  │ false   │ NULL       │ false   │
│     3 │ PARENT_AREA_ID            │ JSON    │ false   │ NULL       │ false   │
│     4 │ AREA_SHORT_CODE           │ VARCHAR │ false   │ NULL       │ false   │
│     5 │ AREA_LONG_CODE            │ VARCHAR │ false   │ NULL       │ false   │
│     6 │ AREA_NAME                 │ VARCHAR │ false   │ NULL       │ false   │
│     7 │ AREA_DESC                 │ VARCHAR │ false   │ NULL       │ false   │
│     8 │ CLASSIFICATION    

In [14]:
# parks table

parks_table = """
CREATE OR REPLACE TABLE analytics.park_table AS
WITH raw_data AS (
    -- Reads the GeoJSON file and converts it into a nested table structure
    SELECT * FROM read_json_auto('source-files/Parks.geojson')
),
unnested_features AS (
    -- Flattens the 'features' array into individual rows
    SELECT unnest(raw_data.features) AS feature_struct FROM raw_data
)
SELECT
    -- 1. Keep the original, meaningful column names from the properties
    feature_struct.properties._id::BIGINT AS _id,
    feature_struct.properties.LOCATIONID::BIGINT AS LOCATIONID,
    feature_struct.properties.ASSET_ID::BIGINT AS ASSET_ID,
    feature_struct.properties.ASSET_NAME::VARCHAR AS ASSET_NAME,
    feature_struct.properties.TYPE::VARCHAR AS TYPE,
    feature_struct.properties.AMENITIES::VARCHAR AS AMENITIES,
    feature_struct.properties.ADDRESS::VARCHAR AS ADDRESS,
    feature_struct.properties.PHONE::VARCHAR AS PHONE,
    feature_struct.properties.URL::VARCHAR AS URL,

    -- 2. Convert the GeoJSON geometry into a WKT string 
    ST_AsText(ST_GeomFromGeoJSON(to_json(feature_struct.geometry))) AS geometry_wkt,
    
    -- 3. Keep the geometry type for reference 
    feature_struct.geometry.type AS geometry_type
FROM unnested_features;
"""

# FIX: Execute the correct variable
con.sql(parks_table)


In [15]:
# check parks table

con.sql("PRAGMA table_info('analytics.park_table')").show()

┌───────┬───────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │     name      │  type   │ notnull │ dflt_value │   pk    │
│ int32 │    varchar    │ varchar │ boolean │  varchar   │ boolean │
├───────┼───────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ _id           │ BIGINT  │ false   │ NULL       │ false   │
│     1 │ LOCATIONID    │ BIGINT  │ false   │ NULL       │ false   │
│     2 │ ASSET_ID      │ BIGINT  │ false   │ NULL       │ false   │
│     3 │ ASSET_NAME    │ VARCHAR │ false   │ NULL       │ false   │
│     4 │ TYPE          │ VARCHAR │ false   │ NULL       │ false   │
│     5 │ AMENITIES     │ VARCHAR │ false   │ NULL       │ false   │
│     6 │ ADDRESS       │ VARCHAR │ false   │ NULL       │ false   │
│     7 │ PHONE         │ VARCHAR │ false   │ NULL       │ false   │
│     8 │ URL           │ VARCHAR │ false   │ NULL       │ false   │
│     9 │ geometry_wkt  │ VARCHAR │ false   │ NULL       │ false   │
│    10 │ geometry_type │ VARCHAR 

In [16]:
# add more data to the neighborhoods table 

add_query = """
ALTER TABLE analytics.neighborhoods ADD COLUMN area_sq_meters DOUBLE;

-- Calculate the area for each neighborhood and update the new column
UPDATE analytics.neighborhoods 
SET area_sq_meters = ST_Area(ST_GeomFromText(geometry_wkt));

ALTER TABLE analytics.neighborhoods ADD COLUMN perimeter_meters DOUBLE;
UPDATE analytics.neighborhoods 
SET perimeter_meters = ST_Perimeter(ST_GeomFromText(geometry_wkt));
"""

con.sql(add_query)

In [17]:
con.sql("PRAGMA table_info('analytics.neighborhoods')").show()

┌───────┬───────────────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │           name            │  type   │ notnull │ dflt_value │   pk    │
│ int32 │          varchar          │ varchar │ boolean │  varchar   │ boolean │
├───────┼───────────────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ _id                       │ BIGINT  │ false   │ NULL       │ false   │
│     1 │ AREA_ID                   │ BIGINT  │ false   │ NULL       │ false   │
│     2 │ AREA_ATTR_ID              │ BIGINT  │ false   │ NULL       │ false   │
│     3 │ PARENT_AREA_ID            │ JSON    │ false   │ NULL       │ false   │
│     4 │ AREA_SHORT_CODE           │ VARCHAR │ false   │ NULL       │ false   │
│     5 │ AREA_LONG_CODE            │ VARCHAR │ false   │ NULL       │ false   │
│     6 │ AREA_NAME                 │ VARCHAR │ false   │ NULL       │ false   │
│     7 │ AREA_DESC                 │ VARCHAR │ false   │ NULL       │ false   │
│     8 │ CLASSIFICATION    

In [ ]:
# TBD: do rental feature eng? total_leased_units, avg_lease_rate_all_types ?

In [18]:
# parks table spatial join with neighborhoods 

spatial_join_query = """
CREATE OR REPLACE TABLE analytics.neighborhoods_with_parks_stats AS
SELECT
    t1.*, -- All columns from neighborhoods 
    COALESCE(COUNT(t2.ASSET_ID), 0) AS park_count
FROM 
    analytics.neighborhoods AS t1 
LEFT JOIN 
    analytics.park_table AS t2 
    ON ST_Intersects(ST_GeomFromText(t1.geometry_wkt), ST_GeomFromText(t2.geometry_wkt)) -- spatial join 
GROUP BY 
    t1.*; -- group by all cols in t1"""

con.sql(spatial_join_query)

In [19]:
con.sql("PRAGMA table_info('analytics.neighborhoods_with_parks_stats')").show()

┌───────┬───────────────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │           name            │  type   │ notnull │ dflt_value │   pk    │
│ int32 │          varchar          │ varchar │ boolean │  varchar   │ boolean │
├───────┼───────────────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ _id                       │ BIGINT  │ false   │ NULL       │ false   │
│     1 │ AREA_ID                   │ BIGINT  │ false   │ NULL       │ false   │
│     2 │ AREA_ATTR_ID              │ BIGINT  │ false   │ NULL       │ false   │
│     3 │ PARENT_AREA_ID            │ JSON    │ false   │ NULL       │ false   │
│     4 │ AREA_SHORT_CODE           │ VARCHAR │ false   │ NULL       │ false   │
│     5 │ AREA_LONG_CODE            │ VARCHAR │ false   │ NULL       │ false   │
│     6 │ AREA_NAME                 │ VARCHAR │ false   │ NULL       │ false   │
│     7 │ AREA_DESC                 │ VARCHAR │ false   │ NULL       │ false   │
│     8 │ CLASSIFICATION    

In [20]:
con.sql("SELECT AREA_NAME, Region_classif FROM analytics.neighborhoods_with_parks_stats WHERE Region_classif IS NOT NULL").show()

┌───────────────────────────────────┬────────────────┐
│             AREA_NAME             │ Region_classif │
│              varchar              │    varchar     │
├───────────────────────────────────┼────────────────┤
│ Woburn North                      │ Toronto E09    │
│ West Rouge                        │ Toronto E11    │
│ Annex                             │ Toronto C02    │
│ St.Andrew-Windfields              │ Toronto C12    │
│ Clairlea-Birchmount               │ Toronto E04    │
│ Humber Bay Shores                 │ Toronto W06    │
│ Thistletown-Beaumond Heights      │ Toronto W10    │
│ Harbourfront-CityPlace            │ Toronto C08    │
│ Dufferin Grove                    │ Toronto C01    │
│ East Willowdale                   │ Toronto C14    │
│        ·                          │      ·         │
│        ·                          │      ·         │
│        ·                          │      ·         │
│ L'Amoreaux West                   │ Toronto E05    │
│ Kensingt

In [21]:
# crime table 

crimes_table = """
CREATE OR REPLACE TABLE analytics.crimes_table AS
WITH raw_data AS (
    -- Reads the GeoJSON file and converts it into a nested table structure
    SELECT * FROM read_json_auto('source-files/crimes.geojson')
),
unnested_features AS (
    -- Flattens the 'features' array into individual rows
    SELECT unnest(raw_data.features) AS feature_struct FROM raw_data
)
SELECT
    -- general data 
    feature_struct.properties._id::BIGINT AS _id,
    feature_struct.properties.AREA_NAME::VARCHAR AS AREA_NAME,
    feature_struct.properties.HOOD_ID::BIGINT AS HOOD_ID, -- NOTE: this is float in geojson 
    
    feature_struct.properties.ASSAULT_2014::INT AS ASSAULT_2014,
    feature_struct.properties.ASSAULT_2015::INT AS ASSAULT_2015,
    feature_struct.properties.ASSAULT_2016::INT AS ASSAULT_2016,
    feature_struct.properties.ASSAULT_2017::INT AS ASSAULT_2017,
    feature_struct.properties.ASSAULT_2018::INT AS ASSAULT_2018,
    feature_struct.properties.ASSAULT_2019::INT AS ASSAULT_2019,
    feature_struct.properties.ASSAULT_2020::INT AS ASSAULT_2020,
    feature_struct.properties.ASSAULT_2021::INT AS ASSAULT_2021,
    feature_struct.properties.ASSAULT_2022::INT AS ASSAULT_2022,
    feature_struct.properties.ASSAULT_2023::INT AS ASSAULT_2023,
    feature_struct.properties.ASSAULT_2024::INT AS ASSAULT_2024,

    --- assault rate 
    feature_struct.properties.ASSAULT_RATE_2014::DOUBLE AS ASSAULT_RATE_2014,
    feature_struct.properties.ASSAULT_RATE_2015::DOUBLE AS ASSAULT_RATE_2015,
    feature_struct.properties.ASSAULT_RATE_2016::DOUBLE AS ASSAULT_RATE_2016,
    feature_struct.properties.ASSAULT_RATE_2017::DOUBLE AS ASSAULT_RATE_2017,
    feature_struct.properties.ASSAULT_RATE_2018::DOUBLE AS ASSAULT_RATE_2018,
    feature_struct.properties.ASSAULT_RATE_2019::DOUBLE AS ASSAULT_RATE_2019,
    feature_struct.properties.ASSAULT_RATE_2020::DOUBLE AS ASSAULT_RATE_2020,
    feature_struct.properties.ASSAULT_RATE_2021::DOUBLE AS ASSAULT_RATE_2021,
    feature_struct.properties.ASSAULT_RATE_2022::DOUBLE AS ASSAULT_RATE_2022,
    feature_struct.properties.ASSAULT_RATE_2023::DOUBLE AS ASSAULT_RATE_2023,
    feature_struct.properties.ASSAULT_RATE_2024::DOUBLE AS ASSAULT_RATE_2024,


    -- autotheft 
    feature_struct.properties.AUTOTHEFT_2014::INT AS AUTOTHEFT_2014,
    feature_struct.properties.AUTOTHEFT_2015::INT AS AUTOTHEFT_2015,
    feature_struct.properties.AUTOTHEFT_2016::INT AS AUTOTHEFT_2016,
    feature_struct.properties.AUTOTHEFT_2017::INT AS AUTOTHEFT_2017,
    feature_struct.properties.AUTOTHEFT_2018::INT AS AUTOTHEFT_2018,
    feature_struct.properties.AUTOTHEFT_2019::INT AS AUTOTHEFT_2019,
    feature_struct.properties.AUTOTHEFT_2020::INT AS AUTOTHEFT_2020,
    feature_struct.properties.AUTOTHEFT_2021::INT AS AUTOTHEFT_2021,
    feature_struct.properties.AUTOTHEFT_2022::INT AS AUTOTHEFT_2022,
    feature_struct.properties.AUTOTHEFT_2023::INT AS AUTOTHEFT_2023,
    feature_struct.properties.AUTOTHEFT_2024::INT AS AUTOTHEFT_2024,


    -- autotheft rate 
    feature_struct.properties.AUTOTHEFT_RATE_2014::DOUBLE AS AUTOTHEFT_RATE_2014,
    feature_struct.properties.AUTOTHEFT_RATE_2015::DOUBLE AS AUTOTHEFT_RATE_2015,
    feature_struct.properties.AUTOTHEFT_RATE_2016::DOUBLE AS AUTOTHEFT_RATE_2016,
    feature_struct.properties.AUTOTHEFT_RATE_2017::DOUBLE AS AUTOTHEFT_RATE_2017,
    feature_struct.properties.AUTOTHEFT_RATE_2018::DOUBLE AS AUTOTHEFT_RATE_2018,
    feature_struct.properties.AUTOTHEFT_RATE_2019::DOUBLE AS AUTOTHEFT_RATE_2019,
    feature_struct.properties.AUTOTHEFT_RATE_2020::DOUBLE AS AUTOTHEFT_RATE_2020,
    feature_struct.properties.AUTOTHEFT_RATE_2021::DOUBLE AS AUTOTHEFT_RATE_2021,
    feature_struct.properties.AUTOTHEFT_RATE_2022::DOUBLE AS AUTOTHEFT_RATE_2022,
    feature_struct.properties.AUTOTHEFT_RATE_2023::DOUBLE AS AUTOTHEFT_RATE_2023,
    feature_struct.properties.AUTOTHEFT_RATE_2024::DOUBLE AS AUTOTHEFT_RATE_2024,
    
    -- biketheft 
    feature_struct.properties.BIKETHEFT_2014::INT AS AUTOTHEFT_2014,
    feature_struct.properties.BIKETHEFT_2015::INT AS AUTOTHEFT_2015,
    feature_struct.properties.BIKETHEFT_2015::INT AS AUTOTHEFT_2015,
    feature_struct.properties.BIKETHEFT_2015::INT AS AUTOTHEFT_2015,
    feature_struct.properties.BIKETHEFT_2015::INT AS AUTOTHEFT_2015,
    feature_struct.properties.BIKETHEFT_2015::INT AS AUTOTHEFT_2015,
    feature_struct.properties.BIKETHEFT_2015::INT AS AUTOTHEFT_2015,
    feature_struct.properties.BIKETHEFT_2015::INT AS AUTOTHEFT_2015,
    feature_struct.properties.BIKETHEFT_2015::INT AS AUTOTHEFT_2015,
    feature_struct.properties.BIKETHEFT_2015::INT AS AUTOTHEFT_2015,

    -- biketheft rate 
    feature_struct.properties.BIKETHEFT_RATE_2014::DOUBLE AS BIKETHEFT_RATE_2014,
    feature_struct.properties.BIKETHEFT_RATE_2015::DOUBLE AS BIKETHEFT_RATE_2015,
    feature_struct.properties.BIKETHEFT_RATE_2016::DOUBLE AS BIKETHEFT_RATE_2016,
    feature_struct.properties.BIKETHEFT_RATE_2017::DOUBLE AS BIKETHEFT_RATE_2017,
    feature_struct.properties.BIKETHEFT_RATE_2018::DOUBLE AS BIKETHEFT_RATE_2018,
    feature_struct.properties.BIKETHEFT_RATE_2019::DOUBLE AS BIKETHEFT_RATE_2019,
    feature_struct.properties.BIKETHEFT_RATE_2020::DOUBLE AS BIKETHEFT_RATE_2020,
    feature_struct.properties.BIKETHEFT_RATE_2021::DOUBLE AS BIKETHEFT_RATE_2021,
    feature_struct.properties.BIKETHEFT_RATE_2022::DOUBLE AS BIKETHEFT_RATE_2022,
    feature_struct.properties.BIKETHEFT_RATE_2023::DOUBLE AS BIKETHEFT_RATE_2023,
    feature_struct.properties.BIKETHEFT_RATE_2024::DOUBLE AS BIKETHEFT_RATE_2024,


    -- break enter 
    feature_struct.properties.BREAKENTER_2014::INT AS BREAKENTER_2014,
    feature_struct.properties.BREAKENTER_2015::INT AS BREAKENTER_2015,
    feature_struct.properties.BREAKENTER_2016::INT AS BREAKENTER_2016,
    feature_struct.properties.BREAKENTER_2017::INT AS BREAKENTER_2017,
    feature_struct.properties.BREAKENTER_2018::INT AS BREAKENTER_2018,
    feature_struct.properties.BREAKENTER_2019::INT AS BREAKENTER_2019,
    feature_struct.properties.BREAKENTER_2020::INT AS BREAKENTER_2020,
    feature_struct.properties.BREAKENTER_2021::INT AS BREAKENTER_2021,
    feature_struct.properties.BREAKENTER_2022::INT AS BREAKENTER_2022,
    feature_struct.properties.BREAKENTER_2023::INT AS BREAKENTER_2023,
    feature_struct.properties.BREAKENTER_2024::INT AS BREAKENTER_2024,


    -- break enter rate 
    feature_struct.properties.BREAKENTER_RATE_2014::DOUBLE AS BIKETHEFT_RATE_2014,
    feature_struct.properties.BREAKENTER_RATE_2015::DOUBLE AS BIKETHEFT_RATE_2015,
    feature_struct.properties.BREAKENTER_RATE_2016::DOUBLE AS BIKETHEFT_RATE_2016,
    feature_struct.properties.BREAKENTER_RATE_2017::DOUBLE AS BIKETHEFT_RATE_2017,
    feature_struct.properties.BREAKENTER_RATE_2018::DOUBLE AS BIKETHEFT_RATE_2018,
    feature_struct.properties.BREAKENTER_RATE_2019::DOUBLE AS BIKETHEFT_RATE_2019,
    feature_struct.properties.BREAKENTER_RATE_2020::DOUBLE AS BIKETHEFT_RATE_2020,
    feature_struct.properties.BREAKENTER_RATE_2021::DOUBLE AS BIKETHEFT_RATE_2021,
    feature_struct.properties.BREAKENTER_RATE_2022::DOUBLE AS BIKETHEFT_RATE_2022,
    feature_struct.properties.BREAKENTER_RATE_2023::DOUBLE AS BIKETHEFT_RATE_2023,
    feature_struct.properties.BREAKENTER_RATE_2024::DOUBLE AS BIKETHEFT_RATE_2024,


    -- homicide 
    feature_struct.properties.HOMICIDE_2014::INT AS HOMICIDE_2014,
    feature_struct.properties.HOMICIDE_2015::INT AS HOMICIDE_2015,
    feature_struct.properties.HOMICIDE_2016::INT AS HOMICIDE_2016,
    feature_struct.properties.HOMICIDE_2017::INT AS HOMICIDE_2017,
    feature_struct.properties.HOMICIDE_2018::INT AS HOMICIDE_2018,
    feature_struct.properties.HOMICIDE_2019::INT AS HOMICIDE_2019,
    feature_struct.properties.HOMICIDE_2020::INT AS HOMICIDE_2020,
    feature_struct.properties.HOMICIDE_2021::INT AS HOMICIDE_2021,
    feature_struct.properties.HOMICIDE_2022::INT AS HOMICIDE_2022,
    feature_struct.properties.HOMICIDE_2023::INT AS HOMICIDE_2023,
    feature_struct.properties.HOMICIDE_2024::INT AS HOMICIDE_2024,


    -- homicide rate 
    feature_struct.properties.HOMICIDE_RATE_2014::DOUBLE AS HOMICIDE_RATE_2014,
    feature_struct.properties.HOMICIDE_RATE_2015::DOUBLE AS HOMICIDE_RATE_2015,
    feature_struct.properties.HOMICIDE_RATE_2016::DOUBLE AS HOMICIDE_RATE_2016,
    feature_struct.properties.HOMICIDE_RATE_2017::DOUBLE AS HOMICIDE_RATE_2017,
    feature_struct.properties.HOMICIDE_RATE_2018::DOUBLE AS HOMICIDE_RATE_2018,
    feature_struct.properties.HOMICIDE_RATE_2019::DOUBLE AS HOMICIDE_RATE_2019,
    feature_struct.properties.HOMICIDE_RATE_2020::DOUBLE AS HOMICIDE_RATE_2020,
    feature_struct.properties.HOMICIDE_RATE_2021::DOUBLE AS HOMICIDE_RATE_2021,
    feature_struct.properties.HOMICIDE_RATE_2022::DOUBLE AS HOMICIDE_RATE_2022,
    feature_struct.properties.HOMICIDE_RATE_2023::DOUBLE AS HOMICIDE_RATE_2023,
    feature_struct.properties.HOMICIDE_RATE_2024::DOUBLE AS HOMICIDE_RATE_2024,

    -- robbery 
    feature_struct.properties.ROBBERY_2014::INT AS ROBBERY_2014,
    feature_struct.properties.ROBBERY_2015::INT AS ROBBERY_2015,
    feature_struct.properties.ROBBERY_2016::INT AS ROBBERY_2016,
    feature_struct.properties.ROBBERY_2017::INT AS ROBBERY_2017,
    feature_struct.properties.ROBBERY_2018::INT AS ROBBERY_2018,
    feature_struct.properties.ROBBERY_2019::INT AS ROBBERY_2019,
    feature_struct.properties.ROBBERY_2020::INT AS ROBBERY_2020,
    feature_struct.properties.ROBBERY_2021::INT AS ROBBERY_2021,
    feature_struct.properties.ROBBERY_2022::INT AS ROBBERY_2022,
    feature_struct.properties.ROBBERY_2023::INT AS ROBBERY_2023,
    feature_struct.properties.ROBBERY_2024::INT AS ROBBERY_2024,


    -- robbery rate 
    feature_struct.properties.ROBBERY_RATE_2014::DOUBLE AS ROBBERY_RATE_2014,
    feature_struct.properties.ROBBERY_RATE_2015::DOUBLE AS ROBBERY_RATE_2015,
    feature_struct.properties.ROBBERY_RATE_2016::DOUBLE AS ROBBERY_RATE_2016,
    feature_struct.properties.ROBBERY_RATE_2017::DOUBLE AS ROBBERY_RATE_2017,
    feature_struct.properties.ROBBERY_RATE_2018::DOUBLE AS ROBBERY_RATE_2018,
    feature_struct.properties.ROBBERY_RATE_2019::DOUBLE AS ROBBERY_RATE_2019,
    feature_struct.properties.ROBBERY_RATE_2020::DOUBLE AS ROBBERY_RATE_2020,
    feature_struct.properties.ROBBERY_RATE_2021::DOUBLE AS ROBBERY_RATE_2021,
    feature_struct.properties.ROBBERY_RATE_2022::DOUBLE AS ROBBERY_RATE_2022,
    feature_struct.properties.ROBBERY_RATE_2023::DOUBLE AS ROBBERY_RATE_2023,
    feature_struct.properties.ROBBERY_RATE_2024::DOUBLE AS ROBBERY_RATE_2024,
    

    -- shooting 
    feature_struct.properties.SHOOTING_2014::INT AS SHOOTING_2014,
    feature_struct.properties.SHOOTING_2015::INT AS SHOOTING_2015,
    feature_struct.properties.SHOOTING_2016::INT AS SHOOTING_2016,
    feature_struct.properties.SHOOTING_2017::INT AS SHOOTING_2017,
    feature_struct.properties.SHOOTING_2018::INT AS SHOOTING_2018,
    feature_struct.properties.SHOOTING_2019::INT AS SHOOTING_2019,
    feature_struct.properties.SHOOTING_2020::INT AS SHOOTING_2020,
    feature_struct.properties.SHOOTING_2021::INT AS SHOOTING_2021,
    feature_struct.properties.SHOOTING_2022::INT AS SHOOTING_2022,
    feature_struct.properties.SHOOTING_2023::INT AS SHOOTING_2023,
    feature_struct.properties.SHOOTING_2024::INT AS SHOOTING_2024,


    -- shooting rate 
    feature_struct.properties.SHOOTING_RATE_2014::DOUBLE AS SHOOTING_RATE_2014,
    feature_struct.properties.SHOOTING_RATE_2015::DOUBLE AS SHOOTING_RATE_2015,
    feature_struct.properties.SHOOTING_RATE_2016::DOUBLE AS SHOOTING_RATE_2016,
    feature_struct.properties.SHOOTING_RATE_2017::DOUBLE AS SHOOTING_RATE_2017,
    feature_struct.properties.SHOOTING_RATE_2018::DOUBLE AS SHOOTING_RATE_2018,
    feature_struct.properties.SHOOTING_RATE_2019::DOUBLE AS SHOOTING_RATE_2019,
    feature_struct.properties.SHOOTING_RATE_2020::DOUBLE AS SHOOTING_RATE_2020,
    feature_struct.properties.SHOOTING_RATE_2021::DOUBLE AS SHOOTING_RATE_2021,
    feature_struct.properties.SHOOTING_RATE_2022::DOUBLE AS SHOOTING_RATE_2022,
    feature_struct.properties.SHOOTING_RATE_2023::DOUBLE AS SHOOTING_RATE_2023,
    feature_struct.properties.SHOOTING_RATE_2024::DOUBLE AS SHOOTING_RATE_2024,
    

    -- theft from motor vehicle 
    feature_struct.properties.THEFTFROMMV_2014::INT AS THEFTFROMMV_2014,
    feature_struct.properties.THEFTFROMMV_2015::INT AS THEFTFROMMV_2015,
    feature_struct.properties.THEFTFROMMV_2016::INT AS THEFTFROMMV_2016,
    feature_struct.properties.THEFTFROMMV_2017::INT AS THEFTFROMMV_2017,
    feature_struct.properties.THEFTFROMMV_2018::INT AS THEFTFROMMV_2018,
    feature_struct.properties.THEFTFROMMV_2019::INT AS THEFTFROMMV_2019,
    feature_struct.properties.THEFTFROMMV_2020::INT AS THEFTFROMMV_2020,
    feature_struct.properties.THEFTFROMMV_2021::INT AS THEFTFROMMV_2021,
    feature_struct.properties.THEFTFROMMV_2022::INT AS THEFTFROMMV_2022,
    feature_struct.properties.THEFTFROMMV_2023::INT AS THEFTFROMMV_2023,
    feature_struct.properties.THEFTFROMMV_2024::INT AS THEFTFROMMV_2024,


    -- theft from moving vehicle rate 
    feature_struct.properties.THEFTFROMMV_RATE_2014::DOUBLE AS THEFTFROMMV_RATE_2014,
    feature_struct.properties.THEFTFROMMV_RATE_2015::DOUBLE AS THEFTFROMMV_RATE_2015,
    feature_struct.properties.THEFTFROMMV_RATE_2016::DOUBLE AS THEFTFROMMV_RATE_2016,
    feature_struct.properties.THEFTFROMMV_RATE_2017::DOUBLE AS THEFTFROMMV_RATE_2017,
    feature_struct.properties.THEFTFROMMV_RATE_2018::DOUBLE AS THEFTFROMMV_RATE_2018,
    feature_struct.properties.THEFTFROMMV_RATE_2019::DOUBLE AS THEFTFROMMV_RATE_2019,
    feature_struct.properties.THEFTFROMMV_RATE_2020::DOUBLE AS THEFTFROMMV_RATE_2020,
    feature_struct.properties.THEFTFROMMV_RATE_2021::DOUBLE AS THEFTFROMMV_RATE_2021,
    feature_struct.properties.THEFTFROMMV_RATE_2022::DOUBLE AS THEFTFROMMV_RATE_2022,
    feature_struct.properties.THEFTFROMMV_RATE_2023::DOUBLE AS THEFTFROMMV_RATE_2023,
    feature_struct.properties.THEFTFROMMV_RATE_2024::DOUBLE AS THEFTFROMMV_RATE_2024,

    -- theftover 
    feature_struct.properties.THEFTOVER_2014::INT AS THEFTOVER_2014,
    feature_struct.properties.THEFTOVER_2015::INT AS THEFTOVER_2015,
    feature_struct.properties.THEFTOVER_2016::INT AS THEFTOVER_2016,
    feature_struct.properties.THEFTOVER_2017::INT AS THEFTOVER_2017,
    feature_struct.properties.THEFTOVER_2018::INT AS THEFTOVER_2018,
    feature_struct.properties.THEFTOVER_2019::INT AS THEFTOVER_2019,
    feature_struct.properties.THEFTOVER_2020::INT AS THEFTOVER_2020,
    feature_struct.properties.THEFTOVER_2021::INT AS THEFTOVER_2021,
    feature_struct.properties.THEFTOVER_2022::INT AS THEFTOVER_2022,
    feature_struct.properties.THEFTOVER_2023::INT AS THEFTOVER_2023,
    feature_struct.properties.THEFTOVER_2024::INT AS THEFTOVER_2024,


    -- theftover rate 
    feature_struct.properties.THEFTOVER_RATE_2014::DOUBLE AS THEFTOVER_RATE_2014,
    feature_struct.properties.THEFTOVER_RATE_2015::DOUBLE AS THEFTOVER_RATE_2015,
    feature_struct.properties.THEFTOVER_RATE_2016::DOUBLE AS THEFTOVER_RATE_2016,
    feature_struct.properties.THEFTOVER_RATE_2017::DOUBLE AS THEFTOVER_RATE_2017,
    feature_struct.properties.THEFTOVER_RATE_2018::DOUBLE AS THEFTOVER_RATE_2018,
    feature_struct.properties.THEFTOVER_RATE_2019::DOUBLE AS THEFTOVER_RATE_2019,
    feature_struct.properties.THEFTOVER_RATE_2020::DOUBLE AS THEFTOVER_RATE_2020,
    feature_struct.properties.THEFTOVER_RATE_2021::DOUBLE AS THEFTOVER_RATE_2021,
    feature_struct.properties.THEFTOVER_RATE_2022::DOUBLE AS THEFTOVER_RATE_2022,
    feature_struct.properties.THEFTOVER_RATE_2023::DOUBLE AS THEFTOVER_RATE_2023,
    feature_struct.properties.THEFTOVER_RATE_2024::DOUBLE AS THEFTOVER_RATE_2024,

    
    feature_struct.properties.POPULATION_2024::INT AS POPULATION_2024,

    -- 2. Convert the GeoJSON geometry into a WKT string 
    ST_AsText(ST_GeomFromGeoJSON(to_json(feature_struct.geometry))) AS geometry_wkt,
    
    -- 3. Keep the geometry type for reference 
    feature_struct.geometry.type AS geometry_type
FROM unnested_features;
"""

con.sql(crimes_table)

In [22]:
con.sql("PRAGMA table_info('analytics.crimes_table')").show()

┌───────┬─────────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │        name         │  type   │ notnull │ dflt_value │   pk    │
│ int32 │       varchar       │ varchar │ boolean │  varchar   │ boolean │
├───────┼─────────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ _id                 │ BIGINT  │ false   │ NULL       │ false   │
│     1 │ AREA_NAME           │ VARCHAR │ false   │ NULL       │ false   │
│     2 │ HOOD_ID             │ BIGINT  │ false   │ NULL       │ false   │
│     3 │ ASSAULT_2014        │ INTEGER │ false   │ NULL       │ false   │
│     4 │ ASSAULT_2015        │ INTEGER │ false   │ NULL       │ false   │
│     5 │ ASSAULT_2016        │ INTEGER │ false   │ NULL       │ false   │
│     6 │ ASSAULT_2017        │ INTEGER │ false   │ NULL       │ false   │
│     7 │ ASSAULT_2018        │ INTEGER │ false   │ NULL       │ false   │
│     8 │ ASSAULT_2019        │ INTEGER │ false   │ NULL       │ false   │
│     9 │ ASSAULT_2020   

In [ ]:
# NOTE: an alternative method is to do it by doing a left join with intersection between the geometry of the neighborhoods.
# since the crime dataset contains rows of neighborhoods with geometry_wkt and the crime statistics for that neighborhood area.

In [23]:
# checking whether AREA_NAME works 

check = """
-- This query finds the "intersection" of names
SELECT
    COUNT(DISTINCT t1.AREA_NAME) AS matching_name_count
FROM
    analytics.neighborhoods_with_parks_stats AS t1
INNER JOIN
    analytics.crimes_table AS t2
    -- Use TRIM and UPPER to clean the text keys before matching
    ON TRIM(UPPER(t1.AREA_NAME)) = TRIM(UPPER(t2.AREA_NAME));"""
con.sql(check)

┌─────────────────────┐
│ matching_name_count │
│        int64        │
├─────────────────────┤
│                 158 │
└─────────────────────┘

In [ ]:
# perfect result for matches for crimes and cumulative dataset thus far, we good

In [24]:
# left join neighborhoods_with_park_stats on crimes 
# parks table spatial join with neighborhoods 

join_query = """
CREATE OR REPLACE TABLE analytics.neighborhoods_parks_crimes AS
SELECT
    t1.*, 
    
    t2.* EXCLUDE (_id, HOOD_ID) 
    
FROM 
    analytics.neighborhoods_with_parks_stats AS t1 
LEFT JOIN 
    analytics.crimes_table AS t2 
    
    ON t1.AREA_NAME = t2.AREA_NAME;
"""

con.sql(join_query)
con.sql("PRAGMA table_info('analytics.neighborhoods_parks_crimes')").show()

┌───────┬─────────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │        name         │  type   │ notnull │ dflt_value │   pk    │
│ int32 │       varchar       │ varchar │ boolean │  varchar   │ boolean │
├───────┼─────────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ _id                 │ BIGINT  │ false   │ NULL       │ false   │
│     1 │ AREA_ID             │ BIGINT  │ false   │ NULL       │ false   │
│     2 │ AREA_ATTR_ID        │ BIGINT  │ false   │ NULL       │ false   │
│     3 │ PARENT_AREA_ID      │ JSON    │ false   │ NULL       │ false   │
│     4 │ AREA_SHORT_CODE     │ VARCHAR │ false   │ NULL       │ false   │
│     5 │ AREA_LONG_CODE      │ VARCHAR │ false   │ NULL       │ false   │
│     6 │ AREA_NAME           │ VARCHAR │ false   │ NULL       │ false   │
│     7 │ AREA_DESC           │ VARCHAR │ false   │ NULL       │ false   │
│     8 │ CLASSIFICATION      │ VARCHAR │ false   │ NULL       │ false   │
│     9 │ CLASSIFICATION_

In [25]:
con.sql("SELECT AREA_NAME, Region_classif FROM analytics.neighborhoods_parks_crimes WHERE Region_classif IS NOT NULL").show()

┌───────────────────────────────────┬────────────────┐
│             AREA_NAME             │ Region_classif │
│              varchar              │    varchar     │
├───────────────────────────────────┼────────────────┤
│ South Eglinton-Davisville         │ Toronto C10    │
│ North Toronto                     │ Toronto C04    │
│ Dovercourt Village                │ Toronto W02    │
│ Junction-Wallace Emerson          │ Toronto W02    │
│ Yonge-Bay Corridor                │ Toronto C08    │
│ Bay-Cloverhill                    │ Toronto C08    │
│ Bendale-Glen Andrew               │ Toronto E09    │
│ Downsview                         │ Toronto W05    │
│ Oakdale-Beverley Heights          │ Toronto W05    │
│ Avondale                          │ Toronto C14    │
│    ·                              │      ·         │
│    ·                              │      ·         │
│    ·                              │      ·         │
│ Kingsview Village-The Westway     │ Toronto W09    │
│ Elms-Old

In [26]:
# load shapes.txt 
transits_table = """
CREATE OR REPLACE TABLE analytics.transit_lines AS
SELECT
    shape_id,
    ST_AsText(
        ST_MakeLine(
            list(
                ST_Point(shape_pt_lon, shape_pt_lat) 
                ORDER BY shape_pt_sequence -- Make sure line is drawn correctly
            )
        )
    ) AS geometry_wkt
    
FROM 
    read_csv_auto('source-files/busdata/shapes.txt') 
GROUP BY 
    shape_id;
"""

In [27]:
# build routes and trips, join routes on trips
routes_table = """
CREATE OR REPLACE TABLE analytics.routes AS
SELECT
    route_id::BIGINT AS route_id,
    agency_id::BIGINT AS agency_id,
    route_short_name::VARCHAR AS route_short_name,
    route_long_name::VARCHAR AS route_long_name,
    route_desc::VARCHAR AS route_desc,
    -- IMPORTANT: route_type tells you what kind of service it is (e.g., 3 = Bus)
    route_type::BIGINT AS route_type, 
    route_url::VARCHAR AS route_url,
    route_color::VARCHAR AS route_color,
    route_text_color::VARCHAR AS route_text_color
FROM 
    read_csv_auto('source-files/busdata/routes.txt');
"""

trips_table = """
CREATE OR REPLACE TABLE analytics.trips AS
SELECT
    route_id::BIGINT AS route_id,
    service_id::BIGINT AS service_id,
    trip_id::BIGINT AS trip_id,
    shape_id::BIGINT AS shape_id,
    
    trip_headsign::VARCHAR AS trip_headsign,
    trip_short_name::VARCHAR AS trip_short_name,
    direction_id::BIGINT AS direction_id,
    block_id::BIGINT AS block_id,
    wheelchair_accessible::BIGINT AS wheelchair_accessible,
    bikes_allowed::BIGINT AS bikes_allowed
FROM 
    read_csv_auto('source-files/busdata/trips.txt');
"""

stop_times_table = """
CREATE OR REPLACE TABLE analytics.stop_times AS
SELECT
    trip_id::BIGINT AS trip_id,
    arrival_time::VARCHAR AS arrival_time,
    departure_time::VARCHAR AS departure_time,
    stop_id::VARCHAR AS stop_id,
    stop_sequence::INT as stop_sequence,
    stop_headsign::VARCHAR as stop_headsign,
    pickup_type::INT as pickup_type,
    drop_off_type:: INT as drop_off_type,
    shape_dist_traveled::FLOAT as shape_dist_travelled
FROM 
    read_csv_auto('source-files/busdata/stop_times.txt');
"""

stops_table = """
CREATE OR REPLACE TABLE analytics.stops AS
SELECT
    stop_id::VARCHAR AS stop_id,
    stop_code::VARCHAR AS stop_code, 
    stop_name::VARCHAR AS stop_name,
    stop_desc::VARCHAR AS stop_desc,
    stop_lat::DOUBLE AS stop_lat, 
    stop_lon::DOUBLE AS stop_lon,
    zone_id::VARCHAR AS zone_id,
    stop_url::VARCHAR AS stop_url,
    location_type::INT AS location_type,
    parent_station::VARCHAR AS parent_station,
    stop_timezone::VARCHAR AS stop_timezone,
    wheelchair_boarding::INT AS wheelchair_boarding
FROM 
    read_csv_auto('source-files/busdata/stops.txt');
"""

stops_geometry_query = """
CREATE OR REPLACE TABLE analytics.transit_stops AS
SELECT
    t.* EXCLUDE (stop_lat, stop_lon), -- Keep all columns except raw coordinates
    -- Create the Geometry Object (POINT) and the WKT string
    ST_AsText(ST_Point(t.stop_lon, t.stop_lat)) AS geometry_wkt
FROM 
    analytics.stops AS t;
"""

In [28]:
con.sql(routes_table)
con.sql(transits_table)
con.sql(stop_times_table)
con.sql(transits_table)
con.sql(stops_table)
con.sql(stops_geometry_query)
con.sql(trips_table)

In [29]:
con.sql("PRAGMA table_info('analytics.transit_lines')").show()

┌───────┬──────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │     name     │  type   │ notnull │ dflt_value │   pk    │
│ int32 │   varchar    │ varchar │ boolean │  varchar   │ boolean │
├───────┼──────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ shape_id     │ BIGINT  │ false   │ NULL       │ false   │
│     1 │ geometry_wkt │ VARCHAR │ false   │ NULL       │ false   │
└───────┴──────────────┴─────────┴─────────┴────────────┴─────────┘



In [30]:
con.sql("PRAGMA table_info('analytics.routes')").show()

┌───────┬──────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │       name       │  type   │ notnull │ dflt_value │   pk    │
│ int32 │     varchar      │ varchar │ boolean │  varchar   │ boolean │
├───────┼──────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ route_id         │ BIGINT  │ false   │ NULL       │ false   │
│     1 │ agency_id        │ BIGINT  │ false   │ NULL       │ false   │
│     2 │ route_short_name │ VARCHAR │ false   │ NULL       │ false   │
│     3 │ route_long_name  │ VARCHAR │ false   │ NULL       │ false   │
│     4 │ route_desc       │ VARCHAR │ false   │ NULL       │ false   │
│     5 │ route_type       │ BIGINT  │ false   │ NULL       │ false   │
│     6 │ route_url        │ VARCHAR │ false   │ NULL       │ false   │
│     7 │ route_color      │ VARCHAR │ false   │ NULL       │ false   │
│     8 │ route_text_color │ VARCHAR │ false   │ NULL       │ false   │
└───────┴──────────────────┴─────────┴─────────┴────────────┴───

In [31]:
con.sql("PRAGMA table_info('analytics.trips')").show()

┌───────┬───────────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │         name          │  type   │ notnull │ dflt_value │   pk    │
│ int32 │        varchar        │ varchar │ boolean │  varchar   │ boolean │
├───────┼───────────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ route_id              │ BIGINT  │ false   │ NULL       │ false   │
│     1 │ service_id            │ BIGINT  │ false   │ NULL       │ false   │
│     2 │ trip_id               │ BIGINT  │ false   │ NULL       │ false   │
│     3 │ shape_id              │ BIGINT  │ false   │ NULL       │ false   │
│     4 │ trip_headsign         │ VARCHAR │ false   │ NULL       │ false   │
│     5 │ trip_short_name       │ VARCHAR │ false   │ NULL       │ false   │
│     6 │ direction_id          │ BIGINT  │ false   │ NULL       │ false   │
│     7 │ block_id              │ BIGINT  │ false   │ NULL       │ false   │
│     8 │ wheelchair_accessible │ BIGINT  │ false   │ NULL       │ false   │

In [32]:
con.sql("PRAGMA table_info('analytics.stop_times')").show()

┌───────┬──────────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │         name         │  type   │ notnull │ dflt_value │   pk    │
│ int32 │       varchar        │ varchar │ boolean │  varchar   │ boolean │
├───────┼──────────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ trip_id              │ BIGINT  │ false   │ NULL       │ false   │
│     1 │ arrival_time         │ VARCHAR │ false   │ NULL       │ false   │
│     2 │ departure_time       │ VARCHAR │ false   │ NULL       │ false   │
│     3 │ stop_id              │ VARCHAR │ false   │ NULL       │ false   │
│     4 │ stop_sequence        │ INTEGER │ false   │ NULL       │ false   │
│     5 │ stop_headsign        │ VARCHAR │ false   │ NULL       │ false   │
│     6 │ pickup_type          │ INTEGER │ false   │ NULL       │ false   │
│     7 │ drop_off_type        │ INTEGER │ false   │ NULL       │ false   │
│     8 │ shape_dist_travelled │ FLOAT   │ false   │ NULL       │ false   │
└───────┴───

In [33]:
con.sql("PRAGMA table_info('analytics.stops')").show()

┌───────┬─────────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │        name         │  type   │ notnull │ dflt_value │   pk    │
│ int32 │       varchar       │ varchar │ boolean │  varchar   │ boolean │
├───────┼─────────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ stop_id             │ VARCHAR │ false   │ NULL       │ false   │
│     1 │ stop_code           │ VARCHAR │ false   │ NULL       │ false   │
│     2 │ stop_name           │ VARCHAR │ false   │ NULL       │ false   │
│     3 │ stop_desc           │ VARCHAR │ false   │ NULL       │ false   │
│     4 │ stop_lat            │ DOUBLE  │ false   │ NULL       │ false   │
│     5 │ stop_lon            │ DOUBLE  │ false   │ NULL       │ false   │
│     6 │ zone_id             │ VARCHAR │ false   │ NULL       │ false   │
│     7 │ stop_url            │ VARCHAR │ false   │ NULL       │ false   │
│     8 │ location_type       │ INTEGER │ false   │ NULL       │ false   │
│     9 │ parent_station 

In [34]:
con.sql("""SELECT
    stop_id,
    stop_name,
    -- Display the full WKT string for inspection
    geometry_wkt
FROM analytics.transit_stops
LIMIT 5;""").show()

┌─────────┬────────────────────────────────┬──────────────────────────────┐
│ stop_id │           stop_name            │         geometry_wkt         │
│ varchar │            varchar             │           varchar            │
├─────────┼────────────────────────────────┼──────────────────────────────┤
│ 662     │ Danforth Rd at Kennedy Rd      │ POINT (-79.260939 43.714379) │
│ 929     │ Davenport Rd at Bedford Rd     │ POINT (-79.399659 43.674448) │
│ 940     │ Davenport Rd at Dupont St      │ POINT (-79.401938 43.675511) │
│ 1871    │ Davisville Ave at Cleveland St │ POINT (-79.378112 43.702088) │
│ 11700   │ Disco Rd at Attwell Dr         │ POINT (-79.594843 43.701362) │
└─────────┴────────────────────────────────┴──────────────────────────────┘



In [35]:
# check tables 
con.sql("PRAGMA table_info('analytics.transit_stops')").show()

┌───────┬─────────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │        name         │  type   │ notnull │ dflt_value │   pk    │
│ int32 │       varchar       │ varchar │ boolean │  varchar   │ boolean │
├───────┼─────────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ stop_id             │ VARCHAR │ false   │ NULL       │ false   │
│     1 │ stop_code           │ VARCHAR │ false   │ NULL       │ false   │
│     2 │ stop_name           │ VARCHAR │ false   │ NULL       │ false   │
│     3 │ stop_desc           │ VARCHAR │ false   │ NULL       │ false   │
│     4 │ zone_id             │ VARCHAR │ false   │ NULL       │ false   │
│     5 │ stop_url            │ VARCHAR │ false   │ NULL       │ false   │
│     6 │ location_type       │ INTEGER │ false   │ NULL       │ false   │
│     7 │ parent_station      │ VARCHAR │ false   │ NULL       │ false   │
│     8 │ stop_timezone       │ VARCHAR │ false   │ NULL       │ false   │
│     9 │ wheelchair_boar

In [36]:
# make a line lookup table for routes and trips join 
line_lookup_table = """
CREATE OR REPLACE TABLE analytics.line_lookup AS
SELECT DISTINCT
    t2.shape_id,
    t1.route_short_name -- e.g., '90', '501'
FROM analytics.routes AS t1
JOIN analytics.trips AS t2 ON t1.route_id = t2.route_id;
"""

# make a stop lookup table with stops and stop time joins 
stop_lookup_table = """
CREATE OR REPLACE TABLE analytics.stop_lookup AS
SELECT
    stop_id,
    COUNT(trip_id) AS total_trips_per_day -- frequency of bus stops per day, 1 id is 1 stop
FROM analytics.stop_times
GROUP BY stop_id;
"""

In [37]:
con.sql(line_lookup_table)
con.sql(stop_lookup_table)

In [38]:
con.sql("PRAGMA table_info('analytics.line_lookup')").show()

┌───────┬──────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │       name       │  type   │ notnull │ dflt_value │   pk    │
│ int32 │     varchar      │ varchar │ boolean │  varchar   │ boolean │
├───────┼──────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ shape_id         │ BIGINT  │ false   │ NULL       │ false   │
│     1 │ route_short_name │ VARCHAR │ false   │ NULL       │ false   │
└───────┴──────────────────┴─────────┴─────────┴────────────┴─────────┘



In [39]:
con.sql("PRAGMA table_info('analytics.stop_lookup')").show()

┌───────┬─────────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │        name         │  type   │ notnull │ dflt_value │   pk    │
│ int32 │       varchar       │ varchar │ boolean │  varchar   │ boolean │
├───────┼─────────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ stop_id             │ VARCHAR │ false   │ NULL       │ false   │
│     1 │ total_trips_per_day │ BIGINT  │ false   │ NULL       │ false   │
└───────┴─────────────────────┴─────────┴─────────┴────────────┴─────────┘



In [40]:
base_cols = con.sql("PRAGMA table_info('analytics.neighborhoods_parks_crimes')").df()['name'].tolist()

# 2. Format the list for SQL, ensuring quotes for column names with spaces
t1_col_list_sql = ', '.join([f't1."{col}"' for col in base_cols])

In [41]:
bus_join_stops= f"""
CREATE OR REPLACE TABLE analytics.neighborhood_bus AS
SELECT
    {t1_col_list_sql}, -- all OG cols 

    --- from stops:
    
    COALESCE(COUNT(DISTINCT(t2.stop_id)), 0) AS total_stop_count, -- accessibility from transit stops
    
    COALESCE(AVG(t3.total_trips_per_day), 0) AS avg_stop_frequency, -- average frequency of stops from stop_lookup

    COALESCE(MAX(t3.total_trips_per_day), 0) AS max_stop_frequency, -- max # of stops per day recorded from stop_lookup

FROM 
    analytics.neighborhoods_parks_crimes AS t1

LEFT JOIN analytics.transit_stops AS t2 
    ON ST_Intersects(ST_GeomFromText(t1.geometry_wkt), ST_GeomFromText(t2.geometry_wkt))

LEFT JOIN analytics.stop_lookup AS t3 ON t2.stop_id = t3.stop_id

GROUP BY {t1_col_list_sql}
"""
con.sql(bus_join_stops)

In [42]:
con.sql("PRAGMA table_info('analytics.neighborhood_bus')").show()

┌───────┬─────────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │        name         │  type   │ notnull │ dflt_value │   pk    │
│ int32 │       varchar       │ varchar │ boolean │  varchar   │ boolean │
├───────┼─────────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ _id                 │ BIGINT  │ false   │ NULL       │ false   │
│     1 │ AREA_ID             │ BIGINT  │ false   │ NULL       │ false   │
│     2 │ AREA_ATTR_ID        │ BIGINT  │ false   │ NULL       │ false   │
│     3 │ PARENT_AREA_ID      │ JSON    │ false   │ NULL       │ false   │
│     4 │ AREA_SHORT_CODE     │ VARCHAR │ false   │ NULL       │ false   │
│     5 │ AREA_LONG_CODE      │ VARCHAR │ false   │ NULL       │ false   │
│     6 │ AREA_NAME           │ VARCHAR │ false   │ NULL       │ false   │
│     7 │ AREA_DESC           │ VARCHAR │ false   │ NULL       │ false   │
│     8 │ CLASSIFICATION      │ VARCHAR │ false   │ NULL       │ false   │
│     9 │ CLASSIFICATION_

In [43]:
base_cols = con.sql("PRAGMA table_info('analytics.neighborhood_bus')").df()['name'].tolist()

# 2. Format the list for SQL, ensuring quotes for column names with spaces
t1_col_list_sql = ', '.join([f't1."{col}"' for col in base_cols])

In [44]:
bus_join_shapes = f"""
CREATE OR REPLACE TABLE analytics.neighborhood_final AS
SELECT
    {t1_col_list_sql}, -- all OG cols 

    -- from shapes: 
    
    COALESCE(SUM(
        ST_Length(ST_Intersection(ST_GeomFromText(t1.geometry_wkt), ST_GeomFromText(t4.geometry_wkt)))
    ), 0) AS total_line_length_meters, -- total length of bus line inside neighborhood boundary 
    
    total_line_length_meters / (t1.area_sq_meters + 1.0) AS transit_line_density, -- total length / area

    COALESCE(COUNT(DISTINCT t5.route_short_name), 0) AS distinct_route_count -- number of different routes

FROM 
    analytics.neighborhood_bus AS t1

LEFT JOIN analytics.transit_lines AS t4
    ON ST_Intersects(ST_GeomFromText(t1.geometry_wkt), ST_GeomFromText(t4.geometry_wkt))

LEFT JOIN analytics.line_lookup AS t5 ON t4.shape_id = t5.shape_id

GROUP BY {t1_col_list_sql}
"""

In [45]:
con.sql(bus_join_shapes)

In [46]:
con.sql("PRAGMA table_info('analytics.neighborhood_final')").show()

┌───────┬──────────────────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │           name           │  type   │ notnull │ dflt_value │   pk    │
│ int32 │         varchar          │ varchar │ boolean │  varchar   │ boolean │
├───────┼──────────────────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ _id                      │ BIGINT  │ false   │ NULL       │ false   │
│     1 │ AREA_ID                  │ BIGINT  │ false   │ NULL       │ false   │
│     2 │ AREA_ATTR_ID             │ BIGINT  │ false   │ NULL       │ false   │
│     3 │ PARENT_AREA_ID           │ JSON    │ false   │ NULL       │ false   │
│     4 │ AREA_SHORT_CODE          │ VARCHAR │ false   │ NULL       │ false   │
│     5 │ AREA_LONG_CODE           │ VARCHAR │ false   │ NULL       │ false   │
│     6 │ AREA_NAME                │ VARCHAR │ false   │ NULL       │ false   │
│     7 │ AREA_DESC                │ VARCHAR │ false   │ NULL       │ false   │
│     8 │ CLASSIFICATION           │ VAR

In [47]:
# Verification Check
con.sql("SELECT AREA_NAME, Region_classif FROM analytics.neighborhood_final WHERE Region_classif IS NOT NULL").show()

┌───────────────────────────────────┬────────────────┐
│             AREA_NAME             │ Region_classif │
│              varchar              │    varchar     │
├───────────────────────────────────┼────────────────┤
│ Keelesdale-Eglinton West          │ Toronto W03    │
│ Bridle Path-Sunnybrook-York Mills │ Toronto C12    │
│ Birchcliffe-Cliffside             │ Toronto E06    │
│ Clanton Park                      │ Toronto C06    │
│ North Toronto                     │ Toronto C04    │
│ York University Heights           │ Toronto W05    │
│ High Park-Swansea                 │ Toronto W01    │
│ Yorkdale-Glen Park                │ Toronto W04    │
│ Mimico-Queensway                  │ Toronto W06    │
│ South Eglinton-Davisville         │ Toronto C10    │
│         ·                         │      ·         │
│         ·                         │      ·         │
│         ·                         │      ·         │
│ Yonge-Bay Corridor                │ Toronto C08    │
│ Forest H

In [48]:
# 158 rows of data, 233 columns of features.

neighborhood_df = con.sql("SELECT * FROM analytics.neighborhood_final").df()

# Method 3: Run a specific query (e.g., to inspect duplicates)
duplicates_df = con.sql("""
    SELECT AREA_NAME, COUNT(*) AS count
    FROM analytics.neighborhood_classified
    GROUP BY AREA_NAME
    HAVING COUNT(*) > 1
""").df()


In [49]:
print(duplicates_df.head()) # check duplicates 

Empty DataFrame
Columns: [AREA_NAME, count]
Index: []


In [50]:
# Check the type and display the first few rows
print(type(neighborhood_df))
print(neighborhood_df.head())

<class 'pandas.core.frame.DataFrame'>
   _id  AREA_ID  AREA_ATTR_ID PARENT_AREA_ID AREA_SHORT_CODE AREA_LONG_CODE  \
0   78  2502289      26022804           None             110            110   
1  112  2502255      26022770           None             041            041   
2   37  2502330      26022845           None             122            122   
3  120  2502247      26022762           None             033            033   
4    2  2502365      26022880           None             173            173   

                           AREA_NAME                               AREA_DESC  \
0           Keelesdale-Eglinton West          Keelesdale-Eglinton West (110)   
1  Bridle Path-Sunnybrook-York Mills  Bridle Path-Sunnybrook-York Mills (41)   
2              Birchcliffe-Cliffside             Birchcliffe-Cliffside (122)   
3                       Clanton Park                       Clanton Park (33)   
4                      North Toronto                     North Toronto (173)   

      

In [52]:
# import pandas as pd
# export parquet and csv 
neighborhood_df.to_csv('tor_neighborhood_condorental.csv', index=False)
neighborhood_df.to_parquet('tor_neighborhood_condorental.parquet', index=False)